In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaForSequenceClassification
from transformers import AdamW
import random
import matplotlib.pyplot as plt
from collections import OrderedDict
from scipy.spatial.distance import cosine
from sim_utils import load_examples, Inputexample, CustomTextDataset, freeze_layers, train, test
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig, AutoModel, AutoTokenizer

os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [2]:
!nvidia-smi

Sun May  1 14:41:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:84:00.0 Off |                  N/A |
| 24%   33C    P8    12W / 250W |   2256MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:85:00.0 Off |                  N/A |
| 22%   

In [3]:
N = 10

data = []
labels = []

train_samples = []
train_labels = []

valid_samples = []
valid_labels = []

test_samples = []
test_labels = []

embed_dim = 768
batch_size = 16 
lr=2e-3  # you can adjust 
temp = 0.3  # you can adjust 
lamda = 0.01  # you can adjust  
skip_time = 0 # the number of time that yi not equal to yj in supervised contrastive loss equation 

data_names = ['CLINC150','BANKING77','HWU64'] 
model_names = ['declutr-small']

shot_names = ['train_5','train_10']

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [4]:
for shot_name in shot_names:
    for data_name in data_names:

        path_shot = f'../../../{data_name}/{shot_name}/'

        valid_path = f'../../../{data_name}/valid/'
        test_path = f'../../../{data_name}/test/'


        # load data
        train_samples = load_examples(path_shot)
        valid_samples = load_examples(valid_path)
        test_samples = load_examples(test_path)


        print("===== small train set ====")
        
        data = []
        labels = []

        for i in range(len(train_samples)):
            data.append(train_samples[i].text)
            labels.append(train_samples[i].label)


        train_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)



        print("===== validation set ====")

        data = []
        labels = []

        for i in range(len(valid_samples)):
            data.append(valid_samples[i].text)
            labels.append(valid_samples[i].label)

        valid_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        valid_loader = DataLoader(valid_data,batch_size=batch_size,shuffle=True)



        print("===== test set ====")

        data = []
        labels = []

        for i in range(len(test_samples)):
            data.append(test_samples[i].text)
            labels.append(test_samples[i].label)

        test_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=True)



         # got the number of unique classes from dataset
        num_class = len(np.unique(np.array(labels)))

         # get text label of uniqure classes
        unique_label = np.unique(np.array(labels))

         # map text label to index classes
        label_maps = {unique_label[i]: i for i in range(len(unique_label))}

        print("label_maps :",label_maps)
        print("num_class:",num_class)


        lines = ['test acc']
        for model_name in model_names:



            exp_name = f'{model_name}_lr={lr}_t={temp}_{data_name}_{shot_name}'
            direct_name = f"johngiorgi/{model_name}"

            print("direct_name :",direct_name)
            tokenizer = AutoTokenizer.from_pretrained(direct_name)
            config = AutoConfig.from_pretrained(direct_name)
            config.num_labels = num_class
            simcse = AutoModelForSequenceClassification.from_pretrained(direct_name,config=config)

            simcse = freeze_layers(simcse,freeze_layers_count=12)
            optimizer= AdamW(simcse.parameters(), lr=lr)
            simcse = simcse.to(device)

            train_log, valid_log = train(exp_name,simcse,device,label_maps,optimizer,train_loader,valid_loader,train_data,valid_data,tokenizer,epochs=30)




            PATH = f'../../../fewshot_models/{exp_name}.pth'
            best_model = AutoModelForSequenceClassification.from_pretrained(direct_name,config=config)
            # Model class must be defined somewhere
            best_model.load_state_dict(torch.load(PATH))
            best_model = best_model.to(device)


            test_acc = test(best_model,device,label_maps,test_loader,len(test_data),tokenizer)

            test_acc = 100 * test_acc
            res = f'shot:{shot_name}_data_name:{data_name}_model:{model_name}_test_acc:{str(test_acc)}'
            lines.append(res)


        with open(f'result_{data_name}_{shot_name}_declutr.txt', 'w') as f:
            for line in lines:
                f.write(line)
                f.write('\n')

===== small train set ====
Train on Cross Entropy loss
len of dataset : 750
===== validation set ====
Train on Cross Entropy loss
len of dataset : 3000
===== test set ====
Train on Cross Entropy loss
len of dataset : 4500
label_maps : {'accept_reservations': 0, 'account_blocked': 1, 'alarm': 2, 'application_status': 3, 'apr': 4, 'are_you_a_bot': 5, 'balance': 6, 'bill_balance': 7, 'bill_due': 8, 'book_flight': 9, 'book_hotel': 10, 'calculator': 11, 'calendar': 12, 'calendar_update': 13, 'calories': 14, 'cancel': 15, 'cancel_reservation': 16, 'car_rental': 17, 'card_declined': 18, 'carry_on': 19, 'change_accent': 20, 'change_ai_name': 21, 'change_language': 22, 'change_speed': 23, 'change_user_name': 24, 'change_volume': 25, 'confirm_reservation': 26, 'cook_time': 27, 'credit_limit': 28, 'credit_limit_change': 29, 'credit_score': 30, 'current_location': 31, 'damaged_card': 32, 'date': 33, 'definition': 34, 'direct_deposit': 35, 'directions': 36, 'distance': 37, 'do_you_have_pets': 38, '

Some weights of the model checkpoint at johngiorgi/declutr-small were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at johngiorgi/declutr-small and are newly ini

classifier.dense.weight
classifier.dense.bias
classifier.out_proj.weight
classifier.out_proj.bias
======  Epoch 1 ====== 
 Training Loss: 0.32270092646280923, 		 Training acc: 0.02266666665673256
train correct :  tensor(17, device='cuda:2')
train total : 750
 Validation Loss: 0.2622704379558563, 		 Validation acc: 0.1340000033378601
valid correct :  tensor(402, device='cuda:2')
valid total : 3000
Validation Loss Decreased(inf--->0.262270) 	 Saving The Model
======  Epoch 2 ====== 
 Training Loss: 0.20885945256551106, 		 Training acc: 0.2706666588783264
train correct :  tensor(203, device='cuda:2')
train total : 750
 Validation Loss: 0.15374849621454875, 		 Validation acc: 0.43166667222976685
valid correct :  tensor(1295, device='cuda:2')
valid total : 3000
Validation Loss Decreased(0.262270--->0.153748) 	 Saving The Model
======  Epoch 3 ====== 
 Training Loss: 0.1110111584663391, 		 Training acc: 0.6213333010673523
train correct :  tensor(466, device='cuda:2')
train total : 750
 Valid

Some weights of the model checkpoint at johngiorgi/declutr-small were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at johngiorgi/declutr-small and are newly ini

correct : 3486
total : 4500
===== small train set ====
Train on Cross Entropy loss
len of dataset : 385
===== validation set ====
Train on Cross Entropy loss
len of dataset : 1540
===== test set ====
Train on Cross Entropy loss
len of dataset : 3080
label_maps : {'Refund_not_showing_up': 0, 'activate_my_card': 1, 'age_limit': 2, 'apple_pay_or_google_pay': 3, 'atm_support': 4, 'automatic_top_up': 5, 'balance_not_updated_after_bank_transfer': 6, 'balance_not_updated_after_cheque_or_cash_deposit': 7, 'beneficiary_not_allowed': 8, 'cancel_transfer': 9, 'card_about_to_expire': 10, 'card_acceptance': 11, 'card_arrival': 12, 'card_delivery_estimate': 13, 'card_linking': 14, 'card_not_working': 15, 'card_payment_fee_charged': 16, 'card_payment_not_recognised': 17, 'card_payment_wrong_exchange_rate': 18, 'card_swallowed': 19, 'cash_withdrawal_charge': 20, 'cash_withdrawal_not_recognised': 21, 'change_pin': 22, 'compromised_card': 23, 'contactless_not_working': 24, 'country_support': 25, 'declin

...

correct : 2303
total : 3080
===== small train set ====
Train on Cross Entropy loss
len of dataset : 640
===== validation set ====
Train on Cross Entropy loss
len of dataset : 1076
===== test set ====
Train on Cross Entropy loss
len of dataset : 1076
label_maps : {'alarm_query': 0, 'alarm_remove': 1, 'alarm_set': 2, 'audio_volume_down': 3, 'audio_volume_mute': 4, 'audio_volume_up': 5, 'calendar_query': 6, 'calendar_remove': 7, 'calendar_set': 8, 'cooking_recipe': 9, 'datetime_convert': 10, 'datetime_query': 11, 'email_addcontact': 12, 'email_query': 13, 'email_querycontact': 14, 'email_sendemail': 15, 'general_affirm': 16, 'general_commandstop': 17, 'general_confirm': 18, 'general_dontcare': 19, 'general_explain': 20, 'general_joke': 21, 'general_negate': 22, 'general_praise': 23, 'general_quirky': 24, 'general_repeat': 25, 'iot_cleaning': 26, 'iot_coffee': 27, 'iot_hue_lightchange': 28, 'iot_hue_lightdim': 29, 'iot_hue_lightoff': 30, 'iot_hue_lighton': 31, 'iot_hue_lightup': 32, 'iot_w

Some weights of the model checkpoint at johngiorgi/declutr-small were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at johngiorgi/declutr-small and are newly ini

classifier.dense.weight
classifier.dense.bias
classifier.out_proj.weight
classifier.out_proj.bias
======  Epoch 1 ====== 
 Training Loss: 0.24030275046825408, 		 Training acc: 0.109375
train correct :  tensor(70, device='cuda:2')
train total : 640
 Validation Loss: 0.1704591086141239, 		 Validation acc: 0.36245352029800415
valid correct :  tensor(390, device='cuda:2')
valid total : 1076
Validation Loss Decreased(inf--->0.170459) 	 Saving The Model
======  Epoch 2 ====== 
 Training Loss: 0.12561222687363624, 		 Training acc: 0.512499988079071
train correct :  tensor(328, device='cuda:2')
train total : 640
 Validation Loss: 0.0951480295467554, 		 Validation acc: 0.5873605608940125
valid correct :  tensor(632, device='cuda:2')
valid total : 1076
Validation Loss Decreased(0.170459--->0.095148) 	 Saving The Model
======  Epoch 3 ====== 
 Training Loss: 0.07561023319140077, 		 Training acc: 0.690625011920929
train correct :  tensor(442, device='cuda:2')
train total : 640
 Validation Loss: 0.

Some weights of the model checkpoint at johngiorgi/declutr-small were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at johngiorgi/declutr-small and are newly ini

correct : 749
total : 1076
